In [1]:
import pandas as pd
df = pd.read_csv("downloads/60k_core_all.csv")

In [2]:
!pip install bertopic
!pip uninstall umap
!pip install umap-learn
!pip install stop-words

     |████████████████████████████████| 57 kB 8.7 MB/s             
     |████████████████████████████████| 57 kB 7.7 MB/s             
     |████████████████████████████████| 55 kB 849 kB/s              
     |████████████████████████████████| 55 kB 6.5 MB/s              
     |████████████████████████████████| 53 kB 5.1 MB/s              
     |████████████████████████████████| 53 kB 3.7 MB/s              
     |████████████████████████████████| 40 kB 1.7 MB/s             
     |████████████▋                   | 348.7 MB 135.8 MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████▏       | 664.9 MB 143.8 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |█████████████████████████████   | 798.9 MB 143.0 MB/s eta 0:00:01

...

     |████████████████████████████████| 881.9 MB 6.4 kB/s              
     |████████████████████████████████| 79 kB 345 kB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 86 kB 295 kB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 5.2 MB 70.8 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 6.4 MB 90.5 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 3.8 MB 109.5 MB/s            
     |████████████████████████████████| 23.3 MB 22.4 MB/s            
     |████████████████████████████████| 1.2 MB 105.6 MB/s            
     |████████████████████████████████| 67 kB 992 kB/s              
     |█████████████████

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████▉       | 683.1 MB 137.5 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 7.5 kB/s              
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp36-cp36m-linux_x86_64.whl size=805388 sha256=0f60dbef69f251c5ae39010223d314c4b1fd5a9a4e72dcc1530ee5895eb844f0
  Stored in directory: /home/ec2-user/.cache/pip/wheels/70/52/fe/83ef5a0e879c48df9968d931d41046226e4cb19a166f0223d2
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=36dad4e767b556943a543a1f4719940464b49cc5e6e976eb36db6c1fbf9e6e32
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ce/2d/86/18ade2fbe66e17694cf8f5667a68d2bd06a65bce42546ea675
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82696 sha256=063f51203b497276ecbffa370d72e390a1106d98e8d428428565d2458c6eb6c8
  Stored in directory: /home/ec2-user/.cache/pip/wheels/21/13/d2/20d08b12788c5c75e5e02cba181b9c8f1e4cdc8616aabf9209
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl

## BERTOPIC

In [2]:
from bertopic import BERTopic
m = BERTopic(embedding_model='sentence-transformers/all-mpnet-base-v2', calculate_probabilities=True, verbose=True)

In [3]:
def extract_topics(texts, distance_thresh=0.035, model=m):
    topic_labels = model.transform(texts)[0]
    topic_dict = {}
    for idx, ID in enumerate(topic_labels):
        topics = model.get_topic(ID)
        for t in topics:
            topic = t[0]
            dist = t[1]
            if dist > distance_thresh:
                if not topic_dict.get(topic):
                    topic_dict[topic] = [ texts[idx] ] 
                else:
                    sents = topic_dict[topic] + [texts[idx]]
                    topic_dict[topic] = sents
    return topic_dict

In [32]:
from stop_words import get_stop_words
def get_topic_name(topic_id, m, take=3,):
    try:
        ts = m.get_topic(topic_id)
        ts = [word[0] for word in ts if word not in get_stop_words('english')]

        name = ""
        for t in ts[:take]:
            name += t
            name += " "
        return name.strip()
    except Exception as e:
        print("Except", topic_id, e)
        return None

In [46]:
!ls downloads/

25k_enriched_small.csv	60k_core_all.csv  core_pos_min.csv  df1.csv  df2.csv


In [5]:
df=df.sample(frac=1)

In [47]:
df1 = df[40000:]

In [48]:
sents1= list(df1["text"])

In [49]:
topics, probs = m.fit_transform(sents1)
#4:27

Batches:   0%|          | 0/830 [00:00<?, ?it/s]

2022-03-07 23:15:38,200 - BERTopic - Transformed documents to Embeddings
2022-03-07 23:15:59,403 - BERTopic - Reduced dimensionality with UMAP
2022-03-07 23:22:21,221 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [50]:
topics[:2]

[5, 223]

In [51]:
get_topic_name(22, m, 3)
#tuberculosis tb mycobacterium'

'listeria monocytogenes listeriosis'

In [52]:
import numpy as np
df1["probs"] = [list(reversed(list(np.argpartition(ps, -3)[-3:]))) for ps in probs]
df1["topic_id"] = topics
df1["topic_name"] = df1["topic_id"].apply(lambda t: get_topic_name(t,m, 3))
df1.to_csv ("downloads/df3.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [54]:
df2 = pd.read_csv("downloads/df2.csv")
df0 = pd.read_csv("downloads/df1.csv")

In [56]:
df = df0.append(df1).append(df2)

In [59]:
df.to_csv ("downloads/65k_enriched_topic.csv", index = False, header=True)

In [12]:
import numpy as np
def stepwise(step, df, f, key):
    l = []
    tn=[]
    for idx in range(step, len(df)+step-1, step):
        m = BERTopic(embedding_model='sentence-transformers/all-mpnet-base-v2', calculate_probabilities=True, verbose=True)
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx][key]
        batch = m.fit_transform(list(sl))
        l = l + list([batch])
#         print(batch[1])
        probs = [list(reversed(list(np.argpartition(ps, -3)[-3:]))) for ps in batch[1]]
        t_names = [get_topic_name(p[0], m, 3) for p in probs]
        tn += [t_names]
    return l, tn

In [15]:
#batches, t_names = stepwise(100, df[:300], m.fit_transform, "text")

In [80]:
def flatten(l):
    return [item for sublist in l for item in sublist]


In [ ]:
t =[]
ps = []
for batch in batches:
    #print(batch[0])
    t += [batch[0]]
    ps += [batch[1]]
#        a+=[t]

a = [flatten(t), flatten(ps)]
len(a), len(a[0])

(2, 66547)

In [ ]:
topics,probs = a

In [ ]:
# td = extract_topics(sentences)

In [94]:
import numpy as np

In [102]:
df["probs"].map(type).unique()

array([<class 'list'>], dtype=object)

In [104]:
m.get_topic()

False

In [101]:

[word[0] for word in m.get_topic(14) if word not in get_stop_words('english')]

['amyotrophic',
 'als',
 'sclerosis',
 'neuron',
 'neurons',
 'neurodegenerative',
 'spinal',
 'degeneration',
 'death',
 'neurone']

In [237]:
df = df.rename(columns={"sentence_id": "problem_id"})

In [2]:
df

,first_sent,doi,coreId,oai,title
0,Liu and Mertz (Genes Dev.,10.1128/MCB.25.15.6303-6313.2005,3717551,oai:pubmedcentral.nih.gov:1190326,Binding of hnRNP L to the Pre-mRNA Processing ...
1,"Rett syndrome (RTT), a neurodevelopmental diso...",10.1086/382228,3710391,oai:pubmedcentral.nih.gov:1182264,X-Chromosome Inactivation Patterns Are Unbalan...
2,Estimating optimal sample size for microbiolog...,10.1128/AEM.69.10.6174-6178.2003,3286476,oai:pubmedcentral.nih.gov:201250,A Model To Estimate the Optimal Sample Size fo...
3,The antiphospholipid syndrome is a thrombotic ...,NaN,8064782,oai:pubmedcentral.nih.gov:2398067,Antiphospholipid syndrome in a patient with ra...
4,Alcohol dependence (AD) is a complex disorder ...,NaN,8674979,oai:pubmedcentral.nih.gov:3266269,Genome-Wide Association Study of Copy Number V...
...,...,...,...,...,...
42196,Autism spectrum disorder (ASD) is a common neu...,NaN,8273706,oai:pubmedcentral.nih.gov:2662567,The genetic and neurobiologic compass points t...
42197,This paper revisits the problem of the readine...,NaN,8689613,oai:pubmedcentral.nih.gov:3291942,On the Relevance of Using Open Wireless Sensor...
42198,Thyroid cancer is a rare disease.,NaN,8126095,oai:pubmedcentral.nih.gov:2466601,Thyroid carcinoma and its treatment
42199,Hyperosmolar hyperglycaemic state (HHS) is a m...,NaN,8696563,oai:pubmedcentral.nih.gov:3302717,A case of hyperosmolar hyperglycaemic state wi...
